In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('diabetic_data.csv')

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        101766 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  101766 non-null object
medical_specialty           101766 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            10176

At face value, it appears that there are no variables for which there are missing values; however, I find this extremely
unlikely. Let's take a look at the top few rows of each variable, to get a sense for how the data is entered and for 
any cases of missing values.

## Categorical data

In [4]:
cat_df = df.select_dtypes(include=['object'])
for col in cat_df.columns:
    print(cat_df[col][0:10])

0          Caucasian
1          Caucasian
2    AfricanAmerican
3          Caucasian
4          Caucasian
5          Caucasian
6          Caucasian
7          Caucasian
8          Caucasian
9          Caucasian
Name: race, dtype: object
0    Female
1    Female
2    Female
3      Male
4      Male
5      Male
6      Male
7      Male
8    Female
9    Female
Name: gender, dtype: object
0      [0-10)
1     [10-20)
2     [20-30)
3     [30-40)
4     [40-50)
5     [50-60)
6     [60-70)
7     [70-80)
8     [80-90)
9    [90-100)
Name: age, dtype: object
0    ?
1    ?
2    ?
3    ?
4    ?
5    ?
6    ?
7    ?
8    ?
9    ?
Name: weight, dtype: object
0    ?
1    ?
2    ?
3    ?
4    ?
5    ?
6    ?
7    ?
8    ?
9    ?
Name: payer_code, dtype: object
0    Pediatrics-Endocrinology
1                           ?
2                           ?
3                           ?
4                           ?
5                           ?
6                           ?
7                           ?
8          

As we can see, there are a few columns for which there is missing data, which is entered as a '?' in this case. Before we decide what to do with missing data, however, it might be worth considering just how bad the problem is for each variable.

In [5]:
for col in cat_df.columns:
    if len(cat_df[cat_df[col]=='?'])*100/101766 > 0: #Only print variables which have more than one missing value
        print('The percentage of missing data for variable {} is {}%'.format(col, len(cat_df[cat_df[col]=='?'])*100/101766))

The percentage of missing data for variable race is 2.2335554114340743%
The percentage of missing data for variable weight is 96.85847925633315%
The percentage of missing data for variable payer_code is 39.5574160328597%
The percentage of missing data for variable medical_specialty is 49.08220820313268%
The percentage of missing data for variable diag_1 is 0.020635575732562937%
The percentage of missing data for variable diag_2 is 0.3517874339170253%
The percentage of missing data for variable diag_3 is 1.398305917497003%


The problem appears to be extremely severe with regards to weight, with almost all the data missing. In this case I 
would recommend simply discarding the variable as there is little information to be gleaned. Similarly, payer code is unlikely to hold much medical significance, and I will discard it here. 

However, despite its high almost 50%) missing data, I would recommend keeping the medical specialty associated with a given patient as this can constitute valuable information for a model. In this case, I will simply impute 'Unknown' to replace the current '?' value. 

The same approach will be taken for the other variables with missing data, as the problem is relatively small there.

In [6]:
df = df.drop(['weight', 'payer_code'], axis=1)
df = df.replace('?', 'Unknown')

## Re-grouping Diagnoses

Reading the original research paper from which this dataset was derived, the 'diag_1', 'diag_2', and 'diag_3' variables
represent primary, secondary, and additional diagnoses associated with the patient. These variables are entered as 
numbers, but the real meaning of these numbers are categorical associations with disease types. To that end, I will 
recode these variables to make it clearer.

In [7]:
df.loc[df['diag_1'].str.contains('E|V|U', regex=True), 'diag_1'] = '10000'
df.loc[df['diag_1'].str.contains('250'), 'diag_1'] = '250'
df['diag_1'] = df['diag_1'].astype(int)
df.loc[df['diag_2'].str.contains('E|V|U', regex=True), 'diag_2'] = '10000'
df.loc[df['diag_2'].str.contains('250'), 'diag_2'] = '250'
df['diag_2'] = df['diag_2'].astype(int)
df.loc[df['diag_3'].str.contains('E|V|U', regex=True), 'diag_3'] = '10000'
df.loc[df['diag_3'].str.contains('250'), 'diag_3'] = '250'
df.loc[df['diag_3'].str.contains('365'), 'diag_3'] = '360'
df['diag_3'] = df['diag_3'].astype(int)

In [8]:
bins = [0, 251, 280, 398, 460, 520, 580, 630, 680, 710, 740, 760,
        785, 786, 787, 788, 789, 800, 1000, np.inf]
names = ['Neoplasms', 'Diabetes', 'Neoplasms', 'Other', 'Circulatory',
         'Respiratory', 'Digestive', 'Genitourinary', 'Other', 'Neplasms',
         'Musculoskeletal', 'Other', 'Neoplasms', 'Circulatory', 'Respiratory',
         'Digestive', 'Genitourinary', 'Neoplasms', 'Injury']

d = dict(enumerate(names, 1))
d

df['diag_1'] = np.vectorize(d.get)(np.digitize(df['diag_1'], bins))
df['diag_2'] = np.vectorize(d.get)(np.digitize(df['diag_2'], bins))
df['diag_3'] = np.vectorize(d.get)(np.digitize(df['diag_3'], bins))

## Numerical data

In [11]:
num_df = df.select_dtypes(include=['int64', 'float64'])
num_df.isna().sum()

encounter_id                0
patient_nbr                 0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
number_diagnoses            0
dtype: int64

Apparently, there are no instances where there is missing data in numerical data. All the better for us! 

A quick processing step, which we will need later: I will convert the int64 columns to float; also, I will need
to convert the discharge_disposition_id and admission_source_id variables from numeric to categorical.

In [12]:
df['discharge_disposition_id'] = df['discharge_disposition_id'].astype('category')
df['admission_source_id'] = df['admission_source_id'].astype('category')
num_vars = df.select_dtypes(include=['int64', 'float64']).columns
for col in num_vars:
    df[col] = df[col].astype(float)

## Further wrangling

In [13]:
df= df.sort_values(by=['patient_nbr', 'encounter_id'])
print(df.head())

       encounter_id  patient_nbr       race  gender      age  \
4267     24437208.0        135.0  Caucasian  Female  [50-60)   
4780     26264286.0        135.0  Caucasian  Female  [50-60)   
5827     29758806.0        378.0  Caucasian  Female  [50-60)   
67608   189899286.0        729.0  Caucasian  Female  [80-90)   
17494    64331490.0        774.0  Caucasian  Female  [80-90)   

       admission_type_id discharge_disposition_id admission_source_id  \
4267                 2.0                        1                   1   
4780                 1.0                        1                   7   
5827                 3.0                        1                   1   
67608                1.0                        3                   7   
17494                1.0                        1                   7   

       time_in_hospital                medical_specialty     ...      \
4267                8.0                       Cardiology     ...       
4780                3.0  Surgery

From this quick exploration, we can see that there is a situation where there are some patients who have multiple 
encounters. This creates a difficult statistical question, as it essentially creates multiple patients who are almost
100% correlated with another patient (i.e. themselves) with regards to demographic variables. 

What I will do in this case, for the sake of simplicity and as a starting case, is to keep only the first instance of individuals which have multiple hospital visits. Further, I will drop encounter_id and patient_nbr, as these are simply identifiers and not actually a source of predictive value.

In [14]:
df = df.drop_duplicates(subset=['patient_nbr'], keep='first')
df = df.drop(['encounter_id', 'patient_nbr'],axis=1)

For this dataset, I will recode the admissions column to reflect only binary outcomes for readmissions: Yes and No.

In [15]:
binary_df = df.copy()
binary_df['readmitted'] = binary_df['readmitted'].map({'<30':'YES', '>30':'YES', 'NO':'NO'}) 

# Export for future use
binary_df.to_csv('binary_diabetes.csv')